In [ ]:
%matplotlib

import math 
import pywt 
import numpy as np
import os
import csv
import fnmatch
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
absoluteSize = False

In [ ]:
TimeDwellOrig = 800
TimeFixation = 300

In [ ]:
def filterBlinks(pupilData, timeList):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
    
    # blink is every nan value
    # 200 ms (18 samples) before and after the blink will also be removed
    addedBlinkSamples = 18
    
    missingPupilData = dict()
    missingPupilData['Start'] = list()
    missingPupilData['End'] = list()
    
    pupilData_filter = list()
    time_filter = list()
    index_blinkEnd = list()
    
    missingBeginning = 0
    missingEnd = 0

    # First, find the missing values
    pupilInd = -1
    for pupilD in pupilData:
        
        pupilInd = pupilInd + 1
        #print(pupilInd, pupilD)
        #if pupilInd > 6800 and pupilInd < 6900:
        #    print(pupilD)
        #print(pupilInd, 'NEXTTTTTTTTTTTT')
        
        # check if val is nan
        if np.isnan(pupilD):
            # check if first element of list
            if pupilInd == 0:
                # first value is nan
                #print(pupilD, 'Very Start')
                missingBeginning = 1
                missingPupilData['Start'].append(pupilInd)
            
            # check if previous val was nan
            elif np.isnan(pupilData[pupilInd-1]):
                
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    #print(pupilInd, len(pupilData))
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        #print(pupilD, 'End')
                        missingPupilData['End'].append(pupilInd)
                else:
                    # last value is nan
                    #print(pupilD, 'Very End')
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                    
            # if new nan, note the index
            else:
                #print(pupilD, 'Start')
                missingPupilData['Start'].append(pupilInd)
                
                # for single nan values
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        #print(pupilD, 'End')
                        missingPupilData['End'].append(pupilInd)
                else:
                    # last value is nan
                    #print(pupilD, 'Very End')
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                
    # print(len(missingPupilData['Start']), len(missingPupilData['End']))
    #for i in range(0, len(missingPupilData['Start'])):
    #     print(missingPupilData['Start'][i], missingPupilData['End'][i])
    
    # Second, create a list that does not contain missing values or 100ms before and after that
    
    # if nan in first 11 samples, missingBeginning should be 1
    if missingPupilData['Start'][0] <= addedBlinkSamples:
        missingBeginning = 1
    
    # similarly, if nan in last 11 samples, missingEnd should be 1
    if len(pupilData) - missingPupilData['End'][-1] <= addedBlinkSamples:
        missingEnd = 1
    
    if missingBeginning:
        
        # append first data
        for valInd in range(missingPupilData['End'][0] + addedBlinkSamples, missingPupilData['Start'][1] - addedBlinkSamples+1):
            pupilData_filter.append(pupilData[valInd])
            time_filter.append(timeList[valInd])
        index_blinkEnd.append(len(pupilData_filter)-1)
        startAppend = 1
            
    else:
        # append first data
        for valInd in range(0, missingPupilData['Start'][0] - addedBlinkSamples+1):
            #print(valInd)
            pupilData_filter.append(pupilData[valInd])
            time_filter.append(timeList[valInd])
        index_blinkEnd.append(len(pupilData_filter)-1)
        startAppend = 0
    
    if missingEnd:
        endAppend = len(missingPupilData['End']) - 3
    else:
        endAppend = len(missingPupilData['End']) - 2

    notPossibleToAdd = 0
    i = -1
    for i in range(startAppend, endAppend):

        if missingPupilData['End'][i]+addedBlinkSamples > missingPupilData['Start'][i+1]-addedBlinkSamples+1:
            notPossibleToAdd = notPossibleToAdd + missingPupilData['Start'][i+1] - missingPupilData['End'][i] - 2
            #print('not possible to add', missingPupilData['End'][i], missingPupilData['End'][i]+addedBlinkSamples, missingPupilData['Start'][i+1], missingPupilData['Start'][i+1]-addedBlinkSamples+1)
        else:
            for valInd in range(missingPupilData['End'][i]+addedBlinkSamples, missingPupilData['Start'][i+1]-addedBlinkSamples+1):
                #print(valInd)
                pupilData_filter.append(pupilData[valInd])
                time_filter.append(timeList[valInd])
            index_blinkEnd.append(len(pupilData_filter)-1)
                
    if missingEnd:
        if missingPupilData['End'][-2]+addedBlinkSamples > missingPupilData['Start'][-1]-addedBlinkSamples+1:
            notPossibleToAdd = notPossibleToAdd + missingPupilData['Start'][-1] - missingPupilData['End'][-2] - 2
        else:
            for valInd in range(missingPupilData['End'][-2]+addedBlinkSamples, missingPupilData['Start'][-1]-addedBlinkSamples+1):
                pupilData_filter.append(pupilData[valInd])
                time_filter.append(timeList[valInd])
            index_blinkEnd.append(len(pupilData_filter)-1)
    else:
        #print(missingPupilData['End'][-1]+addedBlinkSamples, len(pupilData))
        if missingPupilData['End'][-1]+addedBlinkSamples > len(pupilData):
            notPossibleToAdd = notPossibleToAdd + len(pupilData) - missingPupilData['End'][-1] - 2
        else:                
            for valInd in range(missingPupilData['End'][-1]+addedBlinkSamples, len(pupilData)):
                pupilData_filter.append(pupilData[valInd])
                time_filter.append(timeList[valInd])
            index_blinkEnd.append(len(pupilData_filter)-1)

            
            
    #nMissing = 0
    # count total nan values in data:
#     for i in pupilData:
#         if np.isnan(i):
#             nMissing = nMissing + 1
    
    #print(len(pupilData_filter))
    return pupilData_filter, time_filter, index_blinkEnd

In [ ]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [ ]:
# Create list of list (made of epochs) composed of times when user looked at NextPhrase

def UserLookedAtNewTrial(UserLooksAtKey):
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            #print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                #print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    #print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        #print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    #for i in epochList:
    #    print(i)
    
    return epochList

In [ ]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [ ]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [ ]:
def modmax(d):
    # modulus maxima detection
    
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(0, len(d)):
        m[i] = math.fabs(d[i])
    
    # if value is larger than both neighbours , and strictly
    # larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(0, len(d)):
        ll = m[i-1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    #print(len(t))
    return t

In [ ]:
def ipa(d):
    # compute ipa value of pupil diameter
    IPA = list()
    #print(len(d.pupildata.values))
    # obtain 2-level DWT of pupil diameter signal d
    
    try:
        (cA2,cD2,cD1) = pywt.wavedec(d.pupildata.values,'sym16','per',level=2)
    except ValueError:
        #print(ValueError)
        print('value error in wavedec')
        return
    
    # get signal duration (in seconds)
    tt = ((d.timestamp.values[-1] - d.timestamp.values[0]).item())/1000000000
    
    #print(tt)
    
    # normalize by 1=2j , j = 2 for 2-level DWT
    cA2[:] = [x / math.sqrt(4.0) for x in cA2]
    cD1[:] = [x / math.sqrt(2.0) for x in cD1]
    cD2[:] = [x / math.sqrt(4.0) for x in cD2]
    
    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    #print(len(cD2m))
    
    # threshold using universal threshold l_univ = s*sqrt(2logn)
    # where s is the standard deviation of the noise
    luniv = np.std(cD2m) * math.sqrt(2.0*np.log2(len(cD2m)))
    cD2t = pywt.threshold(cD2m ,luniv,mode="hard")
    
    # compute IPA
    ctr = 0
    for i in range(0, len(cD2t)):
        if math.fabs(cD2t[i]) > 0: ctr += 1
        #IPA = float(ctr)/tt
        # maybe each pupil data has an IPA?
    IPA = (float(ctr)/tt)
    
    return IPA, cD2m, cD2t, cD2, cD1, cA2, tt

In [ ]:
def evaluateEachTrial(trialEpoch, scratchPadList, phraseList, gazeLog, timeGazeLog, ipaTrialDict, index_blinkEnd, subjName, absoluteSize):
    
    typing_mechanism = subjName[-2:]
    
    ipaList = list()
    blinkRateList = list()
    pupilSizeList = list()
    
    # customise for one subj
    if subjName == 'mm_MS':
        trialEpoch.remove(trialEpoch[-1])
   
    trialNum = 0
    
    phraseScratchedList = list()
    phraseToBeScratchedList = list()
    phraseScratchedInd = 0
    phraseToBeScratchedInd = 0
    
    timeStrScratch = [item1[0] for item1 in scratchPadList]
    timeStrToBeScracthed = [item2[0] for item2 in phraseList]
    
    timeScratchList = timeConversion(timeStrScratch)
    timeToBeScratchedList = timeConversion(timeStrToBeScracthed)
    
    rowTimeList = -1
    for timeList in trialEpoch:
        
        rowTimeList = rowTimeList + 1
        
        
        
        
        
        # Extract epoch data from gazelog
        
        time1, t1, t2 = timeList[0][0].partition('+')
        timeTrialStart = (datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        # find the actual start of the fixation
        timeScratchPadActive = timeTrialStart - datetime.timedelta(milliseconds=float(timeList[0][3])) - datetime.timedelta(milliseconds=TimeFixation)
        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timeScratchPadActive)
        
        time2, t1, t2 = timeList[-1][0].partition('+')
        timeTrialEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeTrialEnd)
        
        if timeGazeLogEnd - timeGazeLogStart < datetime.timedelta(seconds = 10):
            continue
        
        
        
        
        # Find number of blinks in the trial
        blinkCountStart = [i for i,v in enumerate(index_blinkEnd) if GazeLogStartInd>v]
        #print(blinkCountStart)
        if len(blinkCountStart) < 1:
            blinkCountStartInd = 0
        else:
            blinkCountStartInd = blinkCountStart[-1]
        #print('blink start thingy', GazeLogStartInd, blinkCountStartInd)  
        blinkCountEnd = [i for i,v in enumerate(index_blinkEnd) if GazeLogEndInd<v]

            
        if len(blinkCountEnd) < 1:
            blinkCountEndInd = index_blinkEnd[-1]
        else:
            blinkCountEndInd = blinkCountEnd[0]
        
        blinkFreqTrial = blinkCountEndInd-blinkCountStartInd
        
        
        
        
        
        # Find the stimPhrases and the phraseTyped
        
        timeOfPhraseScratchedNow, phraseScratchedInd = nearestTimePoint(timeScratchList, timeTrialEnd)
        timeOfPhraseToBeScratchedNow, phraseToBeScratchedInd = nearestTimePoint(timeToBeScratchedList, timeTrialEnd)
        
        #print('phrasetime', timeOfPhraseToBeScratchedNow, timeOfPhraseScratchedNow)
        
        if phraseScratchedInd < 0: # sometimes there are random things looked at, in userkeys but not typed
            phraseScratched = ''
        elif scratchPadList[phraseScratchedInd+1][1] != '': # sometimes scratchpad is updated a slight ms after userkeys
            phraseScratched = scratchPadList[phraseScratchedInd+1][1]
            #print('weird timing:    ', phraseScratched)
        else:
            phraseScratched = scratchPadList[phraseScratchedInd][1]
        
        phraseToBeScratched = phraseList[phraseToBeScratchedInd][1]
        
        
        if 'THE EXPERIMENT IS NOW DONE' in phraseToBeScratched:
            #print('exp done')
            continue
            
            
        
        
        
        
        # find pupil size for the trial
        pupilLog = gazeLog[GazeLogStartInd:GazeLogEndInd]
        
        
        
        
        
        
        # find absolute/relative pupil size for the trial
        if absoluteSize:
            pupilSize = pupilLog
        else:
            # compute reference pupil size
            if len(pupilLog) < 40: 
                pupilRef = np.mean(pupilLog[0:math.ceil(len(pupilLog)/2)])
    
            else: #(400ms of reference data - Per's paper)
                pupilRef = np.mean(pupilLog[0:40])
            
            # compute relative pupil size
            pupilSize = [(v-pupilRef)/pupilRef for i,v in enumerate(pupilLog)]
        
        
        
        
        
        # compute ipa for the trial - after the phrases, to not compute if expt is done
        pupilLog_filter_wTime_Tuple = list(zip(time_filterFormatted, pupilLog))
        pupilLog_pd =  pd.DataFrame(pupilLog_filter_wTime_Tuple, columns=['timestamp','pupildata'])
        ipaVal, coeff_modmax, coeff_hard, coeff_D2, coeff_D1, coeff_A, timePeriodTrial = ipa(pupilLog_pd)
        
        
        
        
        
        ipaList.append(ipaVal)
        blinkRateList.append(blinkFreqTrial/timePeriodTrial)
        #print(np.mean(pupilLog))
        pupilSizeList.append(np.mean(pupilSize))
        trialNum = trialNum + 1
    
    
    
    
    
    totalTrialNumVector = [1, 2, 3, 4, 5, 6]
    for i,v in enumerate(ipaList):
        
        ipaKeyName = 'T'+str(i+1)+'_'+typing_mechanism+'_ipa'
        pupilSizeKeyName = 'T'+str(i+1)+'_'+typing_mechanism+'_pupilSize'
        blinkRateKeyName = 'T'+str(i+1)+'_'+typing_mechanism+'_blinkRate'
        ipaTrialDict[ipaKeyName].append(ipaList[i])
        ipaTrialDict[blinkRateKeyName].append(blinkRateList[i])    
        ipaTrialDict[pupilSizeKeyName].append(pupilSizeList[i])
    print(trialNum)
    
    return ipaTrialDict
    
    

In [ ]:
ipaTrial = dict()
ipaTrial['T1_DT_ipa'] = list()
ipaTrial['T2_DT_ipa'] = list()
ipaTrial['T3_DT_ipa'] = list()
ipaTrial['T4_DT_ipa'] = list()
ipaTrial['T5_DT_ipa'] = list()
ipaTrial['T6_DT_ipa'] = list()

ipaTrial['T1_MS_ipa'] = list()
ipaTrial['T2_MS_ipa'] = list()
ipaTrial['T3_MS_ipa'] = list()
ipaTrial['T4_MS_ipa'] = list()
ipaTrial['T5_MS_ipa'] = list()
ipaTrial['T6_MS_ipa'] = list()

ipaTrial['T1_DT_blinkRate'] = list()
ipaTrial['T2_DT_blinkRate'] = list()
ipaTrial['T3_DT_blinkRate'] = list()
ipaTrial['T4_DT_blinkRate'] = list()
ipaTrial['T5_DT_blinkRate'] = list()
ipaTrial['T6_DT_blinkRate'] = list()

ipaTrial['T1_MS_blinkRate'] = list()
ipaTrial['T2_MS_blinkRate'] = list()
ipaTrial['T3_MS_blinkRate'] = list()
ipaTrial['T4_MS_blinkRate'] = list()
ipaTrial['T5_MS_blinkRate'] = list()
ipaTrial['T6_MS_blinkRate'] = list()

ipaTrial['T1_DT_pupilSize'] = list()
ipaTrial['T2_DT_pupilSize'] = list()
ipaTrial['T3_DT_pupilSize'] = list()
ipaTrial['T4_DT_pupilSize'] = list()
ipaTrial['T5_DT_pupilSize'] = list()
ipaTrial['T6_DT_pupilSize'] = list()

ipaTrial['T1_MS_pupilSize'] = list()
ipaTrial['T2_MS_pupilSize'] = list()
ipaTrial['T3_MS_pupilSize'] = list()
ipaTrial['T4_MS_pupilSize'] = list()
ipaTrial['T5_MS_pupilSize'] = list()
ipaTrial['T6_MS_pupilSize'] = list()


In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'

for root, dirs, subfolder in os.walk(subjName):
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
                        
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:
                
                # compile subject names in a list
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                typing_mechanism = subjName[-2:]
                print(subjName)
                
                # obtain right and left pupil data
                pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                timeList = [item[0] if 'Invalid' not in item else np.nan for item in gazeLog]
                
                # filter the blinks
                pupilLogL_filter, time_filter, index_blinkEnd = filterBlinks(pupilLogL, timeList)
                pupilLogR_filter, time_filter, index_blinkEnd = filterBlinks(pupilLogR, timeList)
                
                #print(index_blinkEnd)
                #print(len(pupilLogL_filter))
                time_filterFormatted = timeConversion(time_filter)
                pupilLog_avg = [(pupilLogL_filter[i] + pupilLogR_filter[i])/2 for i in range(0, len(pupilLogL_filter))]
                
                
                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                trialTime = UserLookedAtNewTrial(userKeysWithTime)
                
                #for i in trialTime:
                #    print(i)
                    
                ipaTrial = evaluateEachTrial(trialTime, scratchPad, stimPhrase, pupilLog_avg, time_filterFormatted, ipaTrial, index_blinkEnd, subjName, absoluteSize)
                
                
                
                

In [ ]:
len(ipaTrial['T6_DT_ipa']) + len(ipaTrial['T6_MS_ipa'])

In [ ]:
fig = plt.figure()

n = -1

x = [1, 2, 3, 4, 5, 6]

ax1 = fig.add_subplot(3,1,1) 
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)

colorList = ['b', 'r']
for i in ['DT', 'MS']:
    firstOne = 0
    n = n + 1
    
    aData = list()
    bData = list()
    cData = list()

    for j in [1, 2, 3, 4, 5, 6]:
        ipaPlotVarName = 'T'+str(j)+'_'+i+'_ipa'
        blinkPlotVarName = 'T'+str(j)+'_'+i+'_blinkRate'
        pupilPlotVarName = 'T'+str(j)+'_'+i+'_pupilSize'
        
        a = ax1.errorbar(j, np.mean(ipaTrial[ipaPlotVarName]), yerr = np.std(ipaTrial[ipaPlotVarName]), color = colorList[n], marker = 'o', elinewidth = 4, alpha = 0.5, capsize=8, markersize = 10)
        a[-1][0].set_linestyle('-.')
        aData.append(np.mean(ipaTrial[ipaPlotVarName]))
        b = ax2.errorbar(j, np.mean(ipaTrial[blinkPlotVarName]), yerr = np.std(ipaTrial[blinkPlotVarName]), color = colorList[n], marker = 'o', elinewidth = 4, alpha = 0.5, capsize=8, markersize = 10)
        b[-1][0].set_linestyle('-.')
        bData.append(np.mean(ipaTrial[blinkPlotVarName]))
        c = ax3.errorbar(j, np.mean(ipaTrial[pupilPlotVarName]), yerr = np.std(ipaTrial[pupilPlotVarName]), color = colorList[n], marker = 'o', elinewidth = 4, alpha = 0.5, capsize=8, markersize = 10)
        c[-1][0].set_linestyle('-.')
        cData.append(np.mean(ipaTrial[pupilPlotVarName]))
        
        if firstOne == 0:
            a.set_label(i)
            print(firstOne)
        firstOne = 1
    
    # join lines of different plots
    ax1.plot(x, aData, color = colorList[n], alpha = 0.3)
    ax2.plot(x, bData, color = colorList[n], alpha = 0.3)
    ax3.plot(x, cData, color = colorList[n], alpha = 0.3)
    
    
ax1.legend()
ax1.set_xlabel('Trial number')
ax2.set_xlabel('Trial number')
ax3.set_xlabel('Trial number')

ax1.set_ylabel('IPA value')
ax2.set_ylabel('Blink rate')
ax3.set_ylabel('Pupil size')
  
ax1.set_title('IPA value')
ax2.set_title('Blink rate')
ax3.set_title('Pupil size')

plt.tight_layout()


In [ ]:

ax.boxplot([ipaDict['C'], ipaDict['INC']], positions= [1, 2])
xTicks = ax.get_xticks()
print(xTicks)
ax.set_xticklabels(['Correct', 'Incorrect'])

ax.set_title('Correct- mean: {0}, std: {1}                      Incorrect- mean:  {2},  std:  {3}'.format(np.round(np.mean(ipaDict['C']), 3), np.round(np.std(ipaDict['C']),3), np.round(np.mean(ipaDict['INC']),3), np.round(np.std(ipaDict['INC']),3)))
ax.set_ylim([0, 0.6])

print(scipy.stats.ttest_ind(ipaDict['C'], ipaDict['INC']))
scipy.stats.ranksums(ipaDict['C'], ipaDict['INC'])


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.hist(ipaDict['C_blinkRate'])
ax.hist(ipaDict['INC_blinkRate'], alpha = 0.2)

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([ipaDict['C_blinkRate'], ipaDict['INC_blinkRate']], positions= [1, 2])
xTicks = ax.get_xticks()
print(xTicks)
ax.set_xticklabels(['Correct blink rate', 'Incorrect blink rate'])

ax.set_title('Correct blink rate - mean: {0}, std: {1}          Incorrect blink rate- mean:  {2},  std:  {3}'.format(np.round(np.mean(ipaDict['C_blinkRate']), 3), np.round(np.std(ipaDict['C_blinkRate']),3), np.round(np.mean(ipaDict['INC_blinkRate']),3), np.round(np.std(ipaDict['INC_blinkRate']),3)))
#ax.set_ylim([0, 0.6])

print(scipy.stats.ttest_ind(ipaDict['C'], ipaDict['INC_blinkRate']))
scipy.stats.ranksums(ipaDict['C'], ipaDict['INC_blinkRate'])


In [ ]:
fig = plt.figure()
axL = fig.add_subplot(2,1,1)
axR = fig.add_subplot(2,1,2)
ylim = [-0.02, 0.2]

xaxis = np.arange(0, len(coeff_left), 1/90)

axL.plot(coeff_left)
xTick = axL.get_xticks()
#axL.set_xticklabels(np.round(xTick*(1/90), 2))
axL.set_ylim(ylim)
index_blinkEndAdjust = [i/4 for i in index_blinkEnd]
#for xPoint in index_blinkEndAdjust:
#    axL.plot([xPoint, xPoint], ylim, '--', 'k', alpha = 0.3)

axR.plot(coeff_right)
axR.set_ylim(ylim)

In [ ]:
a = np.arange(0, len(coeff_left)*(4/90), 4/90)

In [ ]:
def plotCoeff_RL(coeffL, coeffR, title):
    
    fig = plt.figure()
    axL = fig.add_subplot(2,1,1)
    axR = fig.add_subplot(2,1,2)
                
    ylim = [np.min([min(coeffL), min(coeffR)])-0.02, np.max([max(coeffL), max(coeffR)]) + 0.02]

    xaxis = np.arange(0, len(coeffL), 1/90)

    axL.plot(coeffL)
    xTicks = axL.get_xticks()
    xTickLabels = [np.round(i*(4/90), 2) for i in xTicks[1:]]
    axL.set_xticks(xTicks[1:-1])
    axL.set_xticklabels(xTickLabels)
    axL.set_ylim(ylim)
    axL.set_title('subject:  {0},  coefficient for {1},  Left IPA  {2}'.format(subjName, title, np.round(ipa_left,4)))
    axL.set_xlabel('Time [in s]')
                
    axR.plot(coeffR)
    axR.set_ylim(ylim)
    axR.set_xticks(xTicks[1:-1])
    axR.set_xticklabels(xTickLabels)
    axR.set_title('Right IPA  {0}'.format(np.round(ipa_right, 4)))
    axR.set_xlabel('Time [in s]')
                
    plt.tight_layout()

In [ ]:
a = []
if len(a) < 1:
    print('yo')

In [ ]:
a = [2, 3, 5]

In [ ]:
5*a